In [ ]:
#make sure to run this notebook in a trusted workspace/folder
#i have run this notebook on vscode (python ver. 3.11.4)

In [ ]:
from vtk import *
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output

In [ ]:
reader = vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()
data = reader.GetOutput()

In [ ]:
num_points = data.GetNumberOfPoints()

In [ ]:
num_points

In [ ]:
ImageFile = data.GetPointData().GetArray('ImageFile')

In [ ]:
#Creating point coordinate lists
x = []
y = []
z = []
for i in range(num_points):
    points = data.GetPoint(i)
    x.append(points[0])
    y.append(points[1])    
    z.append(points[2])    


In [ ]:
#Creating list of values
all_values = [ImageFile.GetValue(i) for i in range(num_points)]

In [ ]:
# help(go.Isosurface.colorscale)

In [ ]:
#Creating isosurface_viz i.e. the first diagram
isosurface_viz = go.Figure(data = go.Isosurface(
    x = x,
    y = y,
    z = z,
    value = all_values,
    isomin = 0,
    isomax = 0,
    colorscale = 'plasma',   
            #  'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
            #  'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
            #  'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
            #  'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
            #  'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
            #  'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
            #  'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
            #  'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
            #  'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
            #  'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
            #  'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
            #  'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
            #  'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
            #  'ylorrd'
    showscale = False,
    cmin = min(all_values),
    cmax = max(all_values),
    opacity = 1,
))

In [ ]:
#Creating histogram_viz i.e. the second diagram
histogram_viz = go.Figure(data = [go.Histogram(x = all_values, nbinsx = 30)]) #nbinsx=30 according to the sample showed in question
clear_output()

In [ ]:
#Defining reset function
def reset_button(reset):

    slider_bar.value = 0 #isovalue back to 0
    isosurface_viz.update_traces(
        isomin = 0,
        isomax = 0
    )

    histogram_viz.data[0].x = all_values #Histogram on whole data 
    
    clear_output() #Clearing old output
    display(layout) #Displaying slider and reset button
    
    viz = make_subplots(rows=1, cols=2,specs=[[{'type':'surface'},{'type':'bar'}]])
    
    viz.add_trace(isosurface_viz.data[0], row = 1, col = 1)
    viz.add_trace(histogram_viz.data[0], row = 1, col = 2)

    viz.update_layout(
        xaxis_title = 'Vortex scalar values', 
        yaxis_title = 'Frequency',
        scene = dict(
        xaxis = dict(showticklabels = False),
        yaxis = dict(showticklabels = False),
        zaxis = dict(showticklabels = False),
        ))
    
    viz.show()
    

    

In [ ]:
#Defining update function
def update_data(updated):
    
    clear_output() #Clearing old output

    isovalue = updated.new
    isosurface_viz.update_traces(
        isomin = isovalue,
        isomax = isovalue
    )

    histogram_viz.data[0].x = [item for item in all_values if item >= isovalue - 0.25 and item <= isovalue + 0.25]   #Selecting data for histogram according to given condition
        
    display(layout) #Displaying slider and reset button
    
    viz = make_subplots(rows=1, cols=2, specs=[[{'type':'surface'}, {'type':'bar'}]])

    viz.add_trace(isosurface_viz.data[0], row = 1, col = 1)
    viz.add_trace(histogram_viz.data[0], row = 1, col = 2) 

    viz.update_layout(
        xaxis_title = 'Vortex scalar values',
        yaxis_title = 'Frequency',
        scene = dict(
        xaxis = dict(showticklabels = False),
        yaxis = dict(showticklabels = False),
        zaxis = dict(showticklabels = False),
        ))
    
    viz.show()

In [ ]:
#This cell might take a while to execute/update
#If the kernel or IDE crashes. Please restart the whole program (not just the notebook)

#Adding a slider widget
slider_bar = widgets.FloatSlider(value = 0, min = min(all_values), max = max(all_values), step = 0.01, description = 'Isovalue ', continuous_update = False,)
slider_bar.observe(update_data, names = 'value')
slider_bar.layout.width = '400px'

button = widgets.Button(description = "Reset")
layout = widgets.HBox([slider_bar,button]) #Adding slider and reset button to layout
display(layout) #Displaying slider and reset button

button.on_click(reset_button)

viz = make_subplots(rows = 1, cols = 2, specs = [[{'type':'surface'},{'type':'bar'}]])

viz.add_trace(isosurface_viz.data[0], row = 1, col = 1)
viz.add_trace(histogram_viz.data[0], row = 1, col = 2)

viz.update_layout(
    xaxis_title = 'Vortex scalar values',
    yaxis_title = 'Frequency',
    scene = dict(
    xaxis = dict(showticklabels = False),
    yaxis = dict(showticklabels = False),
    zaxis = dict(showticklabels = False),
    ))

viz.show()

